In [15]:
import numpy as np
import pandas as pd
from Climate import Country

In [16]:
def fit(X):
    
    means = []
    for i in range(len(X) // 12):
        means.append(np.mean(X[12*i:(i+1)*12]))
        
    a, b = np.polyfit(np.arange(len(means)), np.array(means), 1)
    
    return a, b

In [20]:
def predict(series, horizon):
    
    a, b = fit(series)
    
    preds = []
    for year in range(1 + horizon // 12):
        for month in range(12, 0, -1):
            preds.append(series[-month]+(year+1)*a)
            
    return np.array(preds[:horizon])

In [21]:
loss = []
years = 10

countries = pd.read_csv('final_data.csv').Country.unique()
for country_name in countries:
    
    country = Country(country_name)
    data = country.data()
    country_loss = {'Continent': country._continent, 'Country': country_name}
    series = data.AverageTemperature.values[:-years*12]
    y = data.AverageTemperature.values[-years*12:]
    y_hat = predict(series, years*12)
    fully = data.AverageTemperature.values[-(years+2)*12:]
    country_loss['Loss'] = np.mean((y-y_hat)**2)
    loss.append(country_loss)
    
df = pd.DataFrame(loss)
df

,Continent,Country,Loss
0,Asia,Afghanistan,3.895431
1,Europe,Albania,4.397558
2,Africa,Algeria,1.450848
3,Oceania,American Samoa,0.480787
4,Europe,Andorra,4.412680
...,...,...,...
177,South America,Uruguay,2.060616
178,Asia,Uzbekistan,10.818384
179,Asia,Yemen,0.606626
180,Africa,Zambia,0.513364


In [22]:
df.to_csv('Baseline.csv', index=False)